In [2]:
import pandas as pd
import numpy as np
import pyxlsb as px
import linecache
import math
import matplotlib.pyplot as plt

import datetime 


import qgrid
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

# 1 Data Preprocessing
## 1.1 Read Files

In [3]:
# --- SAP VA05 File ---
#File path
va_file_dir = r'C:\Users\the7490\MDLZ\Central Analytics Team CAT - Projects\Beta Quadrant\KUNLUN\01.Raw Data\VA05_CSV'

#get file name list
va_file_list = os.listdir(va_file_dir)
new_list = []


for file in va_file_list:
    #construct file path
    file_path = os.path.join(va_file_dir,file)
    
    df = pd.read_csv(file_path , encoding = 'utf-8',thousands= r',')
    
    df['Doc. Date'] = pd.to_datetime(df['Doc. Date'])

    df = df.rename(columns=lambda x: x.strip())
    
    new_list.append(df)
    # print(file)
    
#Combine data
va = pd.concat(new_list)

# --- CRS data ---
file_dir = r'C:\Users\the7490\Documents\01 RDS\data\crs_order'
file_list = os.listdir(file_dir)
new_list = []

for file in file_list:
    file_path = os.path.join(file_dir,file)
    dataframe = pd.read_csv(file_path)
    new_list.append(dataframe)
    
#Combine data
crs = pd.concat(new_list)

# --- Boundary data ---
file_dir = r'C:\Users\the7490\Documents\01 RDS\data\min_max_intentory_date'


file_list = os.listdir(file_dir)
new_list = []

for file in file_list:
    file_path = os.path.join(file_dir,file)
    dataframe = pd.read_csv(file_path)
    new_list.append(dataframe)

    
boundary = pd.concat(new_list)

# SKU mapping - Master
sku_mapping = pd.read_excel(r'C:\Users\the7490\Documents\01 RDS\data\SKU Category Mapping List V2.xlsx', engine = "xlrd")
sku_mapping = sku_mapping[['sku_code','Category']].astype(str)

## Functions

In [4]:
def qual_quant_features(data):
    dtypes = list(map(lambda x:str(x),list(data.dtypes)))
    qualitative = []
    quantitative = []
    for i in range(len(dtypes)):
        if dtypes[i] == 'object':
            qualitative.append(data.columns[i])
        else:
            quantitative.append(data.columns[i])
    return qualitative,quantitative

qualitative,quantitative = qual_quant_features(va)

# Create a function to show result
def result_show(path):
    return qgrid.show_grid(path,show_toolbar = True)

# new
def trim(df):
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip()) #trim
    return df

### 1.2.1 Clean Data - SAP VA05

In [63]:
# Filter out wanted columns 
va_df = va[['Rj','PO Number','Sold-To Pt','Name 1','Material','Description','Plnt','Order Qty','ConfirmQty','Doc. Date','Cust.price','Net price','SaTy']]
va_df = va_df[va_df['Order Qty']>0]
va_df[['Sold-To Pt','Material']] = va_df[['Sold-To Pt','Material']].astype(float).astype(int).astype(str)


# Saty == ZOR  and PO number start with VZO for CRS customer
va_df =  va_df[va_df['SaTy']=='ZOR']
va_df = va_df[va_df['PO Number'].str.contains(r'VZO*')]

# Replace null with 0 for Rj
va_df['Rj'] = va_df['Rj'].fillna(0).astype(int)


# Remove 61 code
va_df = va_df[va_df['Rj']!= 61]

# Set Rj=10 if order_qty > confrimQty and Rj is null
# va_df['Rj'] = va_df[['Order Qty','ConfirmQty','Rj']].apply(lambda x: 10 if x['Rj']== 0 and x['Order Qty']> x['ConfirmQty'] else x['Rj'], axis =1)

# Gourp by Customer, Material, Date to map CRS data and count 
va_df_grouped = va_df[['Order Qty','ConfirmQty','Sold-To Pt','Material','Doc. Date','Rj']].groupby(["Sold-To Pt",'Material','Doc. Date']).agg({'Order Qty':'sum','ConfirmQty':'sum','Rj':'count'}).reset_index()
va_df_grouped.rename(columns ={'Rj':'count'}, inplace = True)


# Get Customer and SKU Info
va_cus = va_df[["Sold-To Pt",'Name 1','Plnt']].drop_duplicates(subset=['Sold-To Pt'],keep='first',inplace=False)
va_df_grouped_cus = pd.merge(va_df_grouped, va_cus, on = 'Sold-To Pt',how = 'left')

va_sku = va_df[["Material",'Description','Net price']].drop_duplicates(subset=['Material'],keep='first',inplace=False)
df_va = pd.merge(va_df_grouped_cus, va_sku, on = 'Material',how = 'left')

# df_va =  pd.merge(df_va, date[['Date','year','week']], left_on = 'Doc. Date',right_on = 'Date',how = 'left') 
df_va['year'] = df_va['Doc. Date'].dt.year.astype(int)
df_va['week'] = df_va['Doc. Date'].apply(lambda x: x.strftime("%W")).astype(int)      
df_va['ds'] = df_va['Doc. Date'].apply(lambda x: x.strftime("%Y%m%d")).astype(str)

In [64]:
df_va['Order Qty'].sum()

34520211.0

### 1.2.2 Clean data - CRS 

In [7]:
# Replace \N with null
crs = crs.replace({r'\N': None})

# Set data type
crs['calc_date'] = pd.to_datetime(crs['calc_date'])

crs[['suggest_box','sale_box_week1','sale_box_week2','sale_box_week3','sale_box_week4','sale_box_week5']] = crs[['suggest_box','sale_box_week1','sale_box_week2','sale_box_week3','sale_box_week4','sale_box_week5']].astype(float)
crs[['sold_to_pt','material','ds']] = crs[['sold_to_pt','material','ds']].astype(str)

In [66]:
crs = crs.drop(columns=['year','week'])

### 1.2.3 Clean data - boundary

In [8]:
boundary.drop_duplicates(subset=['CategoryName','ReceiverCode'], keep='first',inplace=True)
boundary = boundary.drop(columns=['CategoryCode','HotFlag','ReceiverName', 'InsertTime'])
boundary[['ReceiverCode']] = boundary[['ReceiverCode']].astype(str)

#####  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 1.3 Merge tables

In [67]:
# merge crs and va
va_crs = pd.merge(df_va, crs,left_on=['Sold-To Pt','Material','ds'],right_on=['sold_to_pt','material','ds'], how='left')

# merge sku mapping to get category 
va_crs_sku = pd.merge(va_crs, sku_mapping,left_on=['material'],right_on=['sku_code'],how='left')

#  merge boundary
va_crs_sku_bdy = pd.merge(va_crs_sku, boundary,left_on=['sold_to_pt','Category'],right_on=['ReceiverCode','CategoryName'],how='left')

MemoryError: Unable to allocate 1.59 GiB for an array with shape (11, 19418782) and data type float64

In [ ]:
va_crs_sku_bdy['Order Qty'].sum()

## 1.4 Organize data

In [11]:
df_prep = trim(va_crs_sku_bdy)

# Fill NA min as 14 and max as 28
df_prep['MinDays'] = df_prep['MinDays'].replace(np.nan, 14).astype(int)
df_prep['MaxDays'] = df_prep['MaxDays'].replace(np.nan, 28).astype(int)

# keep records of ordered date only
df_prep = df_prep[df_prep['order_qty']>0]

# remove avg_sale < 0
df_prep = df_prep[df_prep['avg_sales_box_day']>0]

# remove duplicated columns
df_prep = df_prep.drop(columns=['Sold-To Pt','Material','sku_code','ReceiverCode','calc_date'])

# set data type and  replace fill na with 0
df_prep[['suggest_box','sale_box_week1','sale_box_week2','sale_box_week3','sale_box_week4','sale_box_week5']] = df_prep[['suggest_box','sale_box_week1','sale_box_week2','sale_box_week3','sale_box_week4','sale_box_week5']].fillna(0).astype(float)

df_prep['sale_month'] = (df_prep['sale_box_week1']+ df_prep['sale_box_week2']+ df_prep['sale_box_week3']+ df_prep['sale_box_week4'])

In [12]:
df_prep['Order Qty'].sum()

27073617.0

In [13]:
result_show(va_crs_sku_bdy)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [35]:
va_crs_sku_bdy[va_crs_sku_bdy['material'].isna()].shape

(88645, 35)

In [36]:
va_crs_sku_bdy.shape

(1180667, 35)

In [38]:
df_va.shape

(1180667, 13)

In [55]:
va_crs_sku_bdy['Material'] = va_crs_sku_bdy['Material'].astype(float).astype(int)

In [56]:
va_crs_sku_bdy['Material'].unique()

array([ 244090,  244336,  244338,  244340,  244344,  357405,  357407,
       4009396, 4010501, 4010502, 4010503, 4010504, 4010995, 4015343,
       4015380, 4015382, 4025396, 4038469, 4040512, 4040514, 4040516,
       4040518, 4042308, 4042314, 4042327, 4042334, 4043243, 4043248,
       4043263, 4043987, 4043988, 4043989, 4043990, 4043997, 4043998,
       4043999, 4044000, 4044001, 4044002, 4044003, 4044005, 4044006,
       4044007, 4044009, 4044011, 4044012, 4044015, 4044016, 4044017,
       4044019, 4044020, 4044025, 4048994, 4054458, 4054459, 4061001,
       4061003, 4061347, 4061348, 4061349, 4061491, 4061492, 4061812,
       4074918, 4074920, 4074922, 4074924, 4077890, 4077896, 4077899,
       4079795, 4080617, 4080620, 4080624, 4081339, 4081342, 4081344,
       4081351, 4081352, 4081353, 4081355, 4081445, 4081447, 4081450,
       4081452, 4082386, 4082481, 4084413, 4084414, 4252074, 4252265,
       4256679, 4257609, 4257684, 4257685, 4257803, 4257804, 4257806,
       4257847, 4258

## 1.5 Tag 

In [14]:
# Apply shortage tag via CRS data including reason code 61 (customer real need)
# SAP data starts from 2020
crs['year'] = crs['calc_date'].dt.year.astype(int)
crs['week'] = crs['calc_date'].apply(lambda x: x.strftime("%W")).astype(int)  
crs_incl_61 = crs

crs_incl_61 = crs_incl_61[['order_qty','pgi_qty','year','week','material']].groupby(['year','week','material']).sum().reset_index()

# CFR < 93% → shorage
crs_incl_61['Shortage_incl_61'] = crs_incl_61[['order_qty','pgi_qty']].apply(lambda x: 'shortage' if x['pgi_qty']/x['order_qty'] < 0.93 else 'normal', axis = 1)

# merge
df_prep = pd.merge(df_prep, crs_incl_61[['year','week','material','Shortage_incl_61']], on = ['year','week','material'],how = 'left')

In [15]:
# Define order and pgi type
def assign_order_type(df):
    order = df['Order Qty']
    inv = df['avail_inventory_box']
    # trs = df['intrans_inventory_box']
    mx = df['MaxDays']
    mi = df['MinDays']
    avg = df['avg_sales_box_day']
    
    if order+inv > mx*avg :
        return 'over'
    elif order+inv < mi*avg :
        return 'under'
    else: # order+inv between(mi*avg, mx*avg)
        return 'normal'
    
def assign_pgi_type(df):
    pgi = df['ConfirmQty']
    inv = df['avail_inventory_box']
    mx = df['MaxDays']
    mi = df['MinDays']
    avg = df['avg_sales_box_day']
    
    if pgi+inv > mx*avg :
        return 'exceeded'
    elif pgi+inv < mi*avg :
        return 'lacking'
    else: # pgi+inv between(mi*avg, mx*avg)
        return 'normal'
    
df_prep['order_type'] = df_prep.apply(assign_order_type,axis=1)
df_prep['pgi_type'] = df_prep.apply(assign_pgi_type,axis=1)

df_prep.rename(columns = {'Shortage_incl_61': 'shortage'}, inplace = True)

In [16]:
df = df_prep

# 2 Allocation 
## 2.1 Calculate Gap and exceed amount for shortage records

In [17]:
# Exceed amount: take both max boundary and following month sale into consideration 
    # exceed box = pgi - avail_inventory - max( MaxBoundary* avg_sales, sales_month)
exceed_pgi_box = []

for ot,pt,stg,order,pgi,inv,mx,sale,avg in zip(df['order_type'],df['pgi_type'],df['shortage'],df['Order Qty'],df['ConfirmQty'],df['avail_inventory_box'],df['MaxDays'],df['sale_month'],df['avg_sales_box_day']):
    if stg == 'shortage' and ot == 'over' and pt == 'exceeded':
        if inv > mx*avg:  # Case a:  consider max boundary
            ex_a = pgi #exceed = pgi 
        elif inv+pgi<=mx*avg:
            ex_a = 0 # no exceed pgi
        else: 
            ex_a = pgi + inv - mx*avg
        if pgi+inv > sale: # Case b: consider sales in following month
            ex_b = pgi+inv-sale # not sell in a month
        elif pgi+inv <= sale: #  pgi+inv <= sale sell in a month
            ex_b = 0
        else:
            ex_b = None
        exceed_pgi_box.append(min(ex_a,ex_b))
    else:
        exceed_pgi_box.append(None)

        
df['exceed_pgi_box'] = exceed_pgi_box 
df['exceed_pgi_box'].sum().astype(int) #3,106,537

3106537

In [18]:
# scenario 1： 
    # normal/under ordering GAP = order - pgi - avail_inventory - transit
    # over orderging and !exceeded pgi type GAP = max boundary * avg_sales, sale_month - pgi - avail_inventory - transit
gap_box_s1 = []

for ot,pt,stg,order,pgi,inv,mx,mi,avg in zip(df['order_type'],df['pgi_type'],df['shortage'],df['Order Qty'],df['ConfirmQty'],df['avail_inventory_box'],df['MaxDays'],df['MinDays'],df['avg_sales_box_day']):
    if stg == 'shortage':
        if ot != 'over':
            gap_box_s1.append(max(order - pgi - inv,0))
        elif ot == 'over' and pt !='exceeded': 
            gap_box_s1.append(max(mx*avg - pgi - inv,0))
        else:        
            gap_box_s1.append(None)
    else:
        gap_box_s1.append(None)
        
df['gap_box_s1'] = gap_box_s1
df['gap_box_s1'].sum().astype(int) # 443,076

443076

In [19]:
# scenario 2： 
    # normal GAP = min*avg - pgi - avail_inventory
    # under ordering GAP = order - pgi - avail_inventory
    # over ordering and !exceeded pgi GAP = min*avg - pgi - avail_inventory
gap_box_s2= []

for ot,pt,stg,order,pgi,inv,mx,mi,avg in zip(df['order_type'],df['pgi_type'],df['shortage'],df['Order Qty'],df['ConfirmQty'],df['avail_inventory_box'],df['MaxDays'],df['MinDays'],df['avg_sales_box_day']):
    if stg == 'shortage' :
        if ot == 'normal':
            gap_box_s2.append(max(mi*avg - pgi - inv,0))
        elif ot == 'under': 
            gap_box_s2.append(max(order-pgi-inv,0))
        elif ot == 'over' and pt !='exceeded': 
            gap_box_s2.append(max(mi*avg - pgi - inv,0))
        else:
            gap_box_s2.append(None)
    else:
        gap_box_s2.append(None)

df['gap_box_s2'] = gap_box_s2
df['gap_box_s2'].sum().astype(int) # 278,806

278806

In [20]:
# scenario 3：OWA/avail_inventory  ==0  
    # normal GAP = min*avg - pgi - avail_inventory
    # under ordering GAP = order - pgi - avail_inventory
    # over ordering and normal pgi GAP = min*avg - pgi - avail_inventory
gap_box_s3= []

for ot,pt,stg,order,pgi,inv,mx,mi,avg in zip(df['order_type'],df['pgi_type'],df['shortage'],df['Order Qty'],df['ConfirmQty'],df['avail_inventory_box'],df['MaxDays'],df['MinDays'],df['avg_sales_box_day']):
    if inv <= 0:
        if stg == 'shortage' :
            if ot == 'normal':
                gap_box_s3.append(max(mi*avg - pgi - inv,0))
            elif ot == 'under': 
                gap_box_s3.append(max(order-pgi-inv,0))
            elif ot == 'over' and pt !='exceeded': 
                gap_box_s3.append(max(mi*avg - pgi - inv,0))
            else:
                gap_box_s3.append(None)
        else:
            gap_box_s3.append(None)
    else:
        gap_box_s3.append(None)

df['gap_box_s3'] = gap_box_s3
df['gap_box_s3'].sum().astype(int) # 65,147

65147

## 2.2 Calculate gap can be fill

In [21]:
# GAP and exceed pgi amount group by year, week, material and plant 
sku_gap_fill = df[['year','week','material','Category','Plnt','gap_box_s1','gap_box_s2','gap_box_s3','exceed_pgi_box']]\
                    .groupby(['year','week','material','Plnt'])\
                    .agg({'gap_box_s1':'sum','gap_box_s2':'sum','gap_box_s3':'sum','exceed_pgi_box':'sum','Category':'count'}).reset_index()
sku_gap_fill.rename(columns ={'Category':'count'}, inplace = True)

In [22]:
# Calculate the gap 
gap_can_be_fill_s1 = []
gap_can_be_fill_s2 = []
gap_can_be_fill_s3 = []

for g1, g2, g3, e in zip(sku_gap_fill['gap_box_s1'],sku_gap_fill['gap_box_s2'],sku_gap_fill['gap_box_s3'],sku_gap_fill['exceed_pgi_box']):
    gap_can_be_fill_s1.append(min(g1,e))
    gap_can_be_fill_s2.append(min(g2,e))
    gap_can_be_fill_s3.append(min(g3,e))
    

sku_gap_fill['gap_can_be_fill_s1'] = gap_can_be_fill_s1
sku_gap_fill['gap_can_be_fill_s2'] = gap_can_be_fill_s2
sku_gap_fill['gap_can_be_fill_s3'] = gap_can_be_fill_s3

In [23]:
# Calculate the gap fill index 
sku_gap_fill['gap_fill_index_s1'] = (sku_gap_fill['gap_can_be_fill_s1']/sku_gap_fill['gap_box_s1']).replace(np.inf, 0)
sku_gap_fill['gap_fill_index_s2'] = (sku_gap_fill['gap_can_be_fill_s2']/sku_gap_fill['gap_box_s2']).replace(np.inf, 0)
sku_gap_fill['gap_fill_index_s3'] = (sku_gap_fill['gap_can_be_fill_s3']/sku_gap_fill['gap_box_s3']).replace(np.inf, 0)

In [24]:
# Map index back to df
df1 = pd.merge(df, sku_gap_fill[['year','week','material','Plnt','gap_fill_index_s1','gap_fill_index_s2','gap_fill_index_s3']],
               on=['year','week','material','Plnt'],how='left')

In [25]:
df['Order Qty'].sum()

27073617.0

In [26]:
df1['Order Qty'].sum()

27073617.0

In [27]:
# Calculate gap can be fill value
gap_box_can_be_fill_s1 = []
gap_box_can_be_fill_s2 = []
gap_box_can_be_fill_s3 = []
gap_can_be_fill_value_s1 = []
gap_can_be_fill_value_s2 = []
gap_can_be_fill_value_s3 = []


for ot,pt,stg,g1,g2,g3,ix1,ix2,ix3,price in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],
                                                df1['gap_box_s1'],df1['gap_box_s2'],df1['gap_box_s3'],
                                                df1['gap_fill_index_s1'],df1['gap_fill_index_s2'],df1['gap_fill_index_s3'],df1['Net price']):
    if stg == 'shortage' and pt != 'exceeded':
        # calculate gap box can be fill
        gap_box_can_be_fill_s1.append(g1*ix1)
        gap_box_can_be_fill_s2.append(g2*ix2)
        gap_box_can_be_fill_s3.append(g3*ix3)
        
        # calculate gap box value
        gap_can_be_fill_value_s1.append(g1*ix1*price)
        gap_can_be_fill_value_s2.append(g2*ix2*price)
        gap_can_be_fill_value_s3.append(g3*ix3*price)
    else:
        gap_box_can_be_fill_s1.append(None)
        gap_box_can_be_fill_s2.append(None)
        gap_box_can_be_fill_s3.append(None)
        
        gap_can_be_fill_value_s1.append(None)
        gap_can_be_fill_value_s2.append(None)
        gap_can_be_fill_value_s3.append(None)
        
df1['gap_box_can_be_fill_s1'] = gap_box_can_be_fill_s1
df1['gap_box_can_be_fill_s2'] = gap_box_can_be_fill_s2
df1['gap_box_can_be_fill_s3'] = gap_box_can_be_fill_s3
df1['gap_can_be_fill_value_s1'] = gap_can_be_fill_value_s1
df1['gap_can_be_fill_value_s2'] = gap_can_be_fill_value_s2
df1['gap_can_be_fill_value_s3'] = gap_can_be_fill_value_s3

# Calculate the sum
df1[['gap_box_can_be_fill_s1','gap_box_can_be_fill_s2','gap_box_can_be_fill_s3',
     'gap_can_be_fill_value_s1','gap_can_be_fill_value_s2','gap_can_be_fill_value_s3']].sum().astype(int)

gap_box_can_be_fill_s1        213561
gap_box_can_be_fill_s2        148980
gap_box_can_be_fill_s3         39377
gap_can_be_fill_value_s1    29384480
gap_can_be_fill_value_s2    20116256
gap_can_be_fill_value_s3     5681241
dtype: int32

## 2.3 Calculate reallocated exceed pgi to fill the gap 

In [28]:
# GAP can be fill and exceed pgi amount group by year, week, material and plant
        # Applied in adjusted order
sku_exceed_reallocate = df1[['year','week','material','Plnt','exceed_pgi_box'
                             ,'gap_box_can_be_fill_s1','gap_box_can_be_fill_s2','gap_box_can_be_fill_s3']]\
                                .groupby(['year','week','material','Plnt'])\
                                 .agg({'gap_box_can_be_fill_s1':'sum','gap_box_can_be_fill_s2':'sum','gap_box_can_be_fill_s3':'sum','exceed_pgi_box':'sum'}).reset_index()

# calculate exceed index 
sku_exceed_reallocate['exceed_fill_index_s1'] = (sku_exceed_reallocate['gap_box_can_be_fill_s1']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)
sku_exceed_reallocate['exceed_fill_index_s2'] = (sku_exceed_reallocate['gap_box_can_be_fill_s2']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)
sku_exceed_reallocate['exceed_fill_index_s3'] = (sku_exceed_reallocate['gap_box_can_be_fill_s3']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)

# Map index back to df1
df1 = pd.merge(df1, sku_exceed_reallocate[['year','week','material','Plnt','exceed_fill_index_s1','exceed_fill_index_s2','exceed_fill_index_s3']],
               on=['year','week','material','Plnt'],how='left')

## 2.4 Calculate adjusted order qty

In [29]:
adjusted_order_qty = []

for ot,pt,stg,order,inv,mx,avg,sale in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],df1['Order Qty'],
                                      df1['avail_inventory_box'],df1['MaxDays'],df1['avg_sales_box_day'],df1['sale_month']):
    if stg == 'shortage' and ot == 'over':
        if inv > mx*avg: # Case 1: conside max boundary
            aj_order_a = 0
        else: # inv <= mx*avg
            aj_order_a = mx*avg - inv
        if order + inv > sale: # Case 2: consider following month sale
            aj_order_b = sale - inv
        else:
            aj_order_b = order
        adjusted_order_qty.append(max(aj_order_a, aj_order_b))
    else:
        adjusted_order_qty.append(order)
        
df1['adjusted_order_qty'] = adjusted_order_qty
df1['adjusted_order_qty'].sum().astype(int) #23,539,577

23539577

## 2.5 Calculate adjusted pgi qty

In [30]:
# fill na with 0 for new cols
qualitative,quantitative = qual_quant_features(df1)
df1[quantitative] = df1[quantitative].fillna(0)

adjusted_confirm_qty_s1 = []
adjusted_confirm_qty_s2 = []
adjusted_confirm_qty_s3 = []

for ot,pt,stg,aj_order,pgi,gap1,gap2,gap3,ex, ex_ix1,ex_ix2,ex_ix3 \
    in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],df1['adjusted_order_qty'],df1['ConfirmQty'],
           df1['gap_box_can_be_fill_s1'],df1['gap_box_can_be_fill_s2'],df1['gap_box_can_be_fill_s3'],
           df1['exceed_pgi_box'],df1['exceed_fill_index_s1'],df1['exceed_fill_index_s2'],df1['exceed_fill_index_s3']):
        
        if stg == 'normal':
            adjusted_confirm_qty_s1.append(pgi)
            adjusted_confirm_qty_s2.append(pgi)
            adjusted_confirm_qty_s3.append(pgi)
            
        elif stg == 'shortage' and pt != 'exceeded':
            adjusted_confirm_qty_s1.append(min(pgi+gap1, aj_order))
            adjusted_confirm_qty_s2.append(min(pgi+gap2, aj_order))
            adjusted_confirm_qty_s3.append(min(pgi+gap3, aj_order))
            
        elif stg == 'shortage' and pt == 'exceeded':
            adjusted_confirm_qty_s1.append(min(pgi-ex*ex_ix1, aj_order))
            adjusted_confirm_qty_s2.append(min(pgi-ex*ex_ix2, aj_order))
            adjusted_confirm_qty_s3.append(min(pgi-ex*ex_ix3, aj_order))
            

df1['adjusted_confirm_qty_s1'] = adjusted_confirm_qty_s1
df1['adjusted_confirm_qty_s2'] = adjusted_confirm_qty_s2
df1['adjusted_confirm_qty_s3'] = adjusted_confirm_qty_s3

df1[['adjusted_confirm_qty_s1','adjusted_confirm_qty_s2','adjusted_confirm_qty_s3']].sum().astype(int)

adjusted_confirm_qty_s1    23079332
adjusted_confirm_qty_s2    23015569
adjusted_confirm_qty_s3    22905781
dtype: int32

# 3 Calculate the benefit

## 3.1 Sales

In [31]:
drive_sale_s1 = []
drive_sale_s2 = []
drive_sale_s3 = []

s1_index = 0.25
s2_index = 0.5
s3_index = 1

for ot,pt,stg,gv1,gv2,gv3 in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],
                                        df1['gap_can_be_fill_value_s1'],df1['gap_can_be_fill_value_s2'],df1['gap_can_be_fill_value_s3']):
        if stg == 'shortage' and pt != 'exceed':
            drive_sale_s1.append(s1_index*gv1)
            drive_sale_s2.append(s2_index*gv2)
            drive_sale_s3.append(s3_index*gv3)
        else: 
            drive_sale_s1.append(None)
            drive_sale_s2.append(None)
            drive_sale_s3.append(None)
            
df1['drive_sale_s1']=drive_sale_s1
df1['drive_sale_s2']=drive_sale_s2
df1['drive_sale_s3']=drive_sale_s3
df1[['drive_sale_s1','drive_sale_s2','drive_sale_s3']].sum().astype(int)

drive_sale_s1     7346120
drive_sale_s2    10058128
drive_sale_s3     5681241
dtype: int32

## 3.2 CFR

In [32]:
df_cfr = df1[['order_type','pgi_type','shortage','Order Qty','ConfirmQty','adjusted_order_qty','adjusted_confirm_qty_s1','adjusted_confirm_qty_s2','adjusted_confirm_qty_s3']]

df_cfr_groupby_shortage = df_cfr.groupby(['shortage']).sum().reset_index()

df_cfr_groupby_shortage['original_cfr'] = df_cfr_groupby_shortage['ConfirmQty']/df_cfr_groupby_shortage['Order Qty']*100
df_cfr_groupby_shortage['adjusted_cfr_s1'] = df_cfr_groupby_shortage['adjusted_confirm_qty_s1']/df_cfr_groupby_shortage['adjusted_order_qty']*100
df_cfr_groupby_shortage['adjusted_cfr_s2'] = df_cfr_groupby_shortage['adjusted_confirm_qty_s2']/df_cfr_groupby_shortage['adjusted_order_qty']*100
df_cfr_groupby_shortage['adjusted_cfr_s3'] = df_cfr_groupby_shortage['adjusted_confirm_qty_s3']/df_cfr_groupby_shortage['adjusted_order_qty']*100

df_cfr_groupby_shortage[['shortage','original_cfr','adjusted_cfr_s1','adjusted_cfr_s2','adjusted_cfr_s3']]

,shortage,original_cfr,adjusted_cfr_s1,adjusted_cfr_s2,adjusted_cfr_s3
0,normal,99.694992,99.694992,99.694992,99.694992
1,shortage,89.336637,93.437503,92.410353,90.641773


In [33]:
#df1.to_csv(r'C:\Users\the7490\Downloads\kunlun_v1.csv',encoding='gbk')